# Test Jupyter Notebook
Test code to move MRP around with rospy.

In [1]:
import rospy
from geometry_msgs.msg import Twist

import tf
from gazebo_msgs.msg import ModelState
from gazebo_msgs.srv import GetModelState
from gazebo_msgs.srv import SetModelState

## Moving MRP around

In [2]:
rospy.init_node('test_node', anonymous=True)

In [7]:
pub = rospy.Publisher('/tb_cmd_vel', Twist, queue_size=10)

message = Twist()
message.linear.x = 0.2

pub.publish(message)

In [6]:
message.linear.x = 0
message.angular.z = -0.3

pub.publish(message)

In [8]:
message.linear.x = 0
message.angular.z = 0

pub.publish(message)

## Moving human actor around

In [3]:
get_state = rospy.ServiceProxy('/gazebo/get_model_state', GetModelState)
set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)

In [8]:
person_state = None
rospy.wait_for_service('/gazebo/get_model_state')

try:
    robot_state = get_state("local_user", "world")
    assert robot_state.success is True
except rospy.ServiceException:
    print("/gazebo/get_model_state service call failed")

position = robot_state.pose.position
orientation = robot_state.pose.orientation
print(position)
print(orientation)

x: 1.2030975710714823e-05
y: 0.0006976893064481412
z: -1.1799545412416457e-06
x: 4.3398091044451023e-07
y: 1.5062771152166896e-06
z: 0.00013370358047506276
w: 0.9999999910604478


In [33]:
# rospy.init_node('set_pose')

state_msg = ModelState()
state_msg.model_name = 'local_user'
state_msg.pose.position.x = 2
state_msg.pose.position.y = 2
state_msg.pose.position.z = 0

theta = 2.5
quaternion = tf.transformations.quaternion_from_euler(0, 0, theta)
state_msg.pose.orientation.x = quaternion[0]
state_msg.pose.orientation.y = quaternion[1]
state_msg.pose.orientation.z = quaternion[2]
state_msg.pose.orientation.w = quaternion[3]

rospy.wait_for_service('/gazebo/set_model_state')
try:
    set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
    resp = set_state(state_msg)
except rospy.ServiceException:
    print ("Service call failed: %s" % str(rospy.ServiceException))

In [31]:
delta_t = -0.01

for _ in range(200):
    state_msg.model_name = 'local_user'
    state_msg.pose.position.x += delta_t
    state_msg.pose.position.y = 0
    state_msg.pose.position.z = 0

    theta += delta_t
    quaternion = tf.transformations.quaternion_from_euler(0, 0, theta)
    state_msg.pose.orientation.x = quaternion[0]
    state_msg.pose.orientation.y = quaternion[1]
    state_msg.pose.orientation.z = quaternion[2]
    state_msg.pose.orientation.w = quaternion[3]

    rospy.wait_for_service('/gazebo/set_model_state')
    try:
        set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
        resp = set_state(state_msg)
    except rospy.ServiceException:
        print ("Service call failed: %s" % str(rospy.ServiceException))

## Animating elevator doors

In [ ]:
get_state = rospy.ServiceProxy('/gazebo/get_model_state', GetModelState)
set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)

In [47]:
def get_model_state_from_response(model_name, model_state_response):
    new_model_state = ModelState()
    new_model_state.model_name = model_name
    
    # copy over post and orientation
    new_model_state.pose = model_state_response.pose
    new_model_state.twist = model_state_response.twist
    
    return new_model_state

In [54]:
# get current door positions
left_door_state_msg = None
right_door_state_msg = None

rospy.wait_for_service('/gazebo/get_model_state')

try:
    left_door_state_msg = get_state('elevator_door_left', 'world')
    right_door_state_msg = get_state('elevator_door_right', 'world')
    
    assert left_door_state_msg.success is True
    assert right_door_state_msg.success is True
    
except rospy.ServiceException:
    print('/gazebo/get_model_state service call failed')

In [55]:
left_door_state = get_model_state_from_response(
    'elevator_door_left', left_door_state_msg)
right_door_state = get_model_state_from_response(
    'elevator_door_right', right_door_state_msg)

In [58]:
delta_t = -0.01

for _ in range(200):
    left_door_state.pose.position.y -= delta_t
    right_door_state.pose.position.y += delta_t

    rospy.wait_for_service('/gazebo/set_model_state')
    try:
        set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
        set_state(left_door_state)
        set_state(right_door_state)
        
    except rospy.ServiceException:
        print ("Service call failed: %s" % str(rospy.ServiceException))